# Clonning national examination council of Tanzania (Necta) ACSEE results to analyze Them

## This algorithm is for edicational purpose only

### imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mongoengine import connect, disconnect
import sys
import json

sys.path.append('..')
from models import *

disconnect()
connect('necta_database_2', host='localhost', port=27017)

In [ ]:

#creating a dataframe table for schools data
schools = School.objects.all()

schools_data = []
for school in schools:
    school_data = {
        'school_name': school.school_name,
        'school_index': school.school_index,
        'region': school.school_region,
    }
    schools_data.append(school_data)
        
schools_dataframe = pd.DataFrame(schools_data)

#appending the GPA data to the schools dataframe
all_gpa = SchoolGPA.objects.all()

for gpa in all_gpa:
    schools_dataframe.loc[schools_dataframe['school_index'] == gpa.school.school_index, f'gpa_{gpa.result_year}'] = gpa.year_gpa
    
#calculating the absolute Rate of the school GPA
schools_dataframe['gpa_rate_4'] = schools_dataframe['gpa_2023'] - schools_dataframe['gpa_2024']
schools_dataframe['gpa_rate_3'] = schools_dataframe['gpa_2022'] - schools_dataframe['gpa_2023']
schools_dataframe['gpa_rate_2'] = schools_dataframe['gpa_2021'] - schools_dataframe['gpa_2022']
schools_dataframe['gpa_rate_1'] = schools_dataframe['gpa_2020'] - schools_dataframe['gpa_2021']

schools_dataframe= schools_dataframe.sort_values(by='gpa_2024', ascending=True, na_position='last')
df = schools_dataframe[schools_dataframe["gpa_2024"] != 0]
print(df.head())
    

## single school trend graph using heatmap

In [ ]:
# Select one school (ensure it's a DataFrame, not a Series)
school_name = "S2476"  # Replace with the school index you want
single_school_data = df[df["school_index"] == school_name].set_index("school_name")[["gpa_2020", "gpa_2021", "gpa_2022", "gpa_2023", "gpa_2024"]]

# Convert to DataFrame with years as the index
heatmap_data = single_school_data # Transpose to have years as rows
heatmap_data.columns = [2020, 2021, 2022, 2023, 2024]
# Plot heatmap
plt.figure(figsize=(4, 6))  # Adjust figure size for single column
sns.heatmap(heatmap_data, cmap="coolwarm", annot=True, fmt=".2f", linewidths=0.5)

# Graph Styling
plt.xlabel("Year")
plt.ylabel("School Name")
plt.title("Top 5 Schools GPA Heatmap (Best GPA = 1, Worst GPA = 5)")
plt.gca().invert_yaxis()  # Best school appears at the top

plt.show()

## Top 10 best Schools

### The following heatmap drawn to indicate the schools GPA over the past four years

In [ ]:
top_10_schools = df.head(10)

# Extract only GPA columns + School Name
heatmap_data = top_10_schools.set_index("school_name")[["gpa_2020", "gpa_2021", "gpa_2022", "gpa_2023", "gpa_2024"]]

# Rename columns for better readability (now in ascending order)
heatmap_data.columns = [2020, 2021, 2022, 2023, 2024]

# Plot heatmap (reverse the color scale)
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, cmap="coolwarm", annot=True, fmt=".2f", linewidths=0.5)

# Graph Styling
plt.xlabel("Year")
plt.ylabel("School Name")
plt.title("Top 10 Schools GPA Heatmap (Best GPA = 1, Worst GPA = 5)")

plt.show()

## top trends schools heatmap

### This part will solve for the schools that have best positive rates, meaning that over the past four years they successfull climbed reaching the high

In [ ]:
#schools that have positive gradual increase in GPA over the years
df["positive_rates_count"] = (df[['gpa_rate_1', 'gpa_rate_2', 'gpa_rate_3', 'gpa_rate_4']] > 0).sum(axis=1)
df_3 = df.sort_values(by=["positive_rates_count", "gpa_2024"], ascending=[False, True], na_position="last")

# Extract only GPA columns + School Name
heatmap_data = top_10_schools.set_index("school_name")[["gpa_2020", "gpa_2021", "gpa_2022", "gpa_2023", "gpa_2024"]]

# Rename columns for better readability (now in ascending order)
heatmap_data.columns = [2020, 2021, 2022, 2023, 2024]

# Plot heatmap (reverse the color scale)
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, cmap="coolwarm", annot=True, fmt=".2f", linewidths=0.5)

# Graph Styling
plt.xlabel("Year")
plt.ylabel("School Name")
plt.title("Top 10 Schools GPA Heatmap (Best GPA = 1, Worst GPA = 5)")

plt.show()

## Subject performance
### This part we are going to cover the performance of each subjects and their correration to the school results

In [ ]:
subjects = SubjectPerformance.objects.all()

school_performance=[]

for subject in subjects:
    school_name = subject.school.school_name
    data = (school_name, subject.school.school_index, subject.subject_name, subject.subject_code, subject.result_year, subject.subject_gpa)
    school_performance.append(data)

#creating index
index = pd.MultiIndex.from_tuples(
    [(school, school_index, subject_name, subjects_code) for school, school_index, subject_name, subjects_code, _, _ in school_performance],
    names=["school_name", "school_index", "subject_name", "subjects_code"]
)

# Create DataFrame
df = pd.DataFrame(
    index=index,
    columns=["gpa_2024", "gpa_2023", "gpa_2022", "gpa_2021", "gpa_2020"]
)


# Fill DataFrame with actual GPA values
for school, school_index, subject_name, subject_code, result_year, subject_gpa in school_performance:
    df.loc[(school, school_index, subject_name, subject_code), f"gpa_{result_year}"] = subject_gpa

# Display DataFrame
df.to_pickle("subject_performance_pickle")
print(df.tail(20))

## calculating the total subject gpa over the past 4 years

In [ ]:
#importing the dataframe from the saved pickle
subject_performance = pd.read_pickle("subject_performance_pickle")

#computing the schools scoring highest gpa per subject
subject_performance['subject_score'] = (subject_performance[["gpa_2024", "gpa_2023", "gpa_2022", "gpa_2021", "gpa_2020"]]).sum(axis=1, skipna=False)

# Reset index to access 'school_name' as a column
subject_performance_reset = subject_performance.reset_index()


combinations = ["PCB", "PCM", "CBG", "HGL", "HGK", "HGE", "EGM"]
#creating the dataframe representing schools and combination scores
combination_dataframe = pd.DataFrame(
    index=subject_performance_reset["school_name"].unique(),
    columns=combinations
)

schools = list(combination_dataframe.index)
for school in schools:
    # Filter data for the current school
    school_data = subject_performance_reset[subject_performance_reset["school_name"] == school].drop_duplicates()

    # Function to compute score only if all subjects exist
    def compute_combination_score(required_subjects):
        available_subjects = set(school_data["subject_name"])
        if required_subjects.issubset(available_subjects):
            return school_data.loc[school_data["subject_name"].isin(required_subjects), "subject_score"].sum(skipna=False)
        return np.nan  # Return NaN if any subject is missing

    # Compute combination scores
    combination_scores = {
        "PCB": compute_combination_score({"PHYSICS", "CHEMISTRY", "BIOLOGY"}),
        "PCM": compute_combination_score({"PHYSICS", "CHEMISTRY", "ADVANCED MATHEMATICS"}),
        "CBG": compute_combination_score({"CHEMISTRY", "BIOLOGY", "GEOGRAPHY"}),
        "HGL": compute_combination_score({"HISTORY", "GEOGRAPHY", "ENGLISH LANGUAGE"}),
        "HGK": compute_combination_score({"HISTORY", "GEOGRAPHY", "KISWAHILI"}),
        "HKL": compute_combination_score({"HISTORY", "KISWAHILI", "ENGLISH LANGUAGE"}),
        "HGE": compute_combination_score({"HISTORY", "GEOGRAPHY", "ECONOMICS"}),
        "EGM": compute_combination_score({"ECONOMICS", "GEOGRAPHY", "ADVANCED MATHEMATICS"})
    }

    # Assign the values to combination_dataframe
    for key, value in combination_scores.items():
        combination_dataframe.loc[school, key] = value

#order the schools based on combination performance, and the lower the score the higher the performance
with open("combinations_performer.txt", "w") as f:
    for comb in combinations:
        # Sort the dataframe based on the current combination
        combination_dataframe = combination_dataframe.sort_values(by=comb, ascending=True, na_position="last")
        
        # Remove rows where the combination score is 0
        combination_dataframe = combination_dataframe[combination_dataframe[comb] != 0]
        result_dataframe = combination_dataframe[[comb]]
        
        # Display top results for debugging
        print(result_dataframe.head())

        # Convert top 5 rows to a string and write to file
        f.write(f"\nTop performers for {comb}:\n")
        f.write(result_dataframe.head(5).to_string(index=True))  # Convert DataFrame to string
        f.write("\n" + "-" * 50 + "\n")

In [ ]:
necta_result = pd.read_pickle("necta_results_pickle")
print(necta_result.head(10))